In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

import torch
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms.functional import normalize
import torch.nn.functional as F

In [2]:
walldata = fetch_openml(name='wall-robot-navigation',version=3)

X_tv, X_test, y_tv, y_test = train_test_split(walldata.data, walldata.target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.2, shuffle=False)


In [3]:
X_wall_train = torch.from_numpy(X_train.to_numpy()).float()
X_wall_val = torch.from_numpy(X_val.to_numpy()).float()
X_wall_test = torch.from_numpy(X_test.to_numpy()).float()
y_wall_train = torch.from_numpy(y_train.to_numpy().astype(int)-1).long()
y_wall_val = torch.from_numpy(y_val.to_numpy().astype(int)-1).long()
y_wall_test = torch.from_numpy(y_test.to_numpy().astype(int)-1).long()


In [4]:
bs = 32
wall_train_ds = TensorDataset(X_wall_train, y_wall_train)
wall_train_dl = DataLoader(wall_train_ds, batch_size=bs)
wall_val_ds = TensorDataset(X_wall_val, y_wall_val)
wall_val_dl = DataLoader(wall_val_ds, batch_size=len(wall_val_ds))
wall_test_ds = TensorDataset(X_wall_test, y_wall_test)
wall_test_dl = DataLoader(wall_test_ds, batch_size=len(wall_test_ds))


In [5]:
indim = 4

hiddim1 = 10
hiddim2 = 10
hiddim3 = 10

outdim = 4

learning_rate = 1e-3
epochs = 500

torch.manual_seed(0)

model = torch.nn.Sequential(
    torch.nn.Linear(indim,hiddim1,bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(hiddim1,hiddim2,bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(hiddim2,hiddim3,bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(hiddim3,outdim,bias=True)
)

In [8]:
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

loss_fn = torch.nn.CrossEntropyLoss()

size = len(wall_train_dl.dataset)



In [ ]:
goodenough = False
goodenoughvalue = 0.99
for epoch in range(epochs):
    for batch, (X, y) in enumerate(wall_train_dl):

        # All the training stuff

        if ((epoch % 1 == 0) and (batch % 50 == 0)) or (loss.item() <= 0.001):
            print("Epoch {} Batch {}: Training Loss = {}: Accuracy = {}".format(epoch,batch,
                                                        loss.item(),correct))
            with torch.no_grad():
                for X, y in wall_val_dl:
                    pred = model(X)
                    loss = loss_fn(pred, y)
                    loss_value = loss.item()
                    
                    y_pred = torch.argmax(pred.data,1)
                    correct = (y == y_pred).sum().item()/len(y_pred)
                    if (epoch % 1 == 0) and (batch % 10 == 0):
                        print("Validation Loss = {}: Accuracy = {}".format(loss_value,correct))
                    if correct >= goodenoughvalue:
                        goodenough = True
        if goodenough:
            break
    if goodenough:
        break


with torch.no_grad():
    for X, y in wall_val_dl:
        pred = model(X)
        loss = loss_fn(pred, y)
        loss_value = loss.item()
        
        y_pred = torch.argmax(pred.data,1)
        correct = (y == y_pred).sum().item()/len(y_pred)

        print("Validation Loss = {}: Accuracy = {}".format(loss_value,correct))


In [ ]:
with torch.no_grad():
    for X, y in wall_test_dl:
        pred = model(X)
        loss = loss_fn(pred, y)
        loss_value = loss.item()
        
        y_pred = torch.argmax(pred.data,1)
        correct = (y == y_pred).sum().item()/len(y_pred)

        print("Test Loss = {}: Accuracy = {}".format(loss_value,correct))


In [ ]:
learning_rate = 1e-3
epochs = 50

outchan1 = 10
kernel1 = 5
outchan2 = 20
kernel2 = 5

# For output sizing
conv1 = torch.nn.Conv2d(1,outchan1,kernel1)
pool = torch.nn.MaxPool2d(2)
conv2 = torch.nn.Conv2d(outchan1,outchan2,kernel2)

print(X_crack_train[0:1].shape)
print(conv1(X_crack_train[0:1]).shape)
print(pool(conv1(X_crack_train[0:1])).shape)
print(F.relu(pool(conv1(X_crack_train[0:1]))).shape)
print(conv2(F.relu(pool(conv1(X_crack_train[0:1])))).shape)
print(pool(conv2(F.relu(pool(conv1(X_crack_train[0:1]))))).shape)

imgout = F.relu(pool(conv2(F.relu(pool(conv1(X_crack_train[0:1]))))))
print(imgout.shape)

infc1 = np.prod(imgout.shape)
print(infc1)
outfc1 = 50
outfc2 = 2

torch.manual_seed(0)


In [ ]:
class CNN(torch.nn.Module):
    def __init__(self,outchan1,kernel1,outchan2,kernel2,infc1,outfc1,outfc2):
        super(CNN,self).__init__()
        self.infc1 = infc1
        self.conv1 = torch.nn.Conv2d(1,outchan1,kernel1)
        self.pool = torch.nn.MaxPool2d(2)
        self.conv2 = torch.nn.Conv2d(outchan1,outchan2,kernel2)
        self.fc1 = torch.nn.Linear(infc1,outfc1)
        self.fc2 = torch.nn.Linear(outfc1,outfc2)
    def forward(self,x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = x.view(-1,self.infc1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = CNN(outchan1,kernel1,outchan2,kernel2,infc1,outfc1,outfc2)
